In [ ]:
import os
import shutil 
from numpy import arange

from utils import *


In [ ]:
'''from google.colab import files

uploaded = files.upload()
deltas, indexes = deltas_in_search('search_J1J-1')
print(deltas)

print(len(deltas))
counter = 1
for k,v in deltas.items():
  counter *= len(v)
  
print(counter)
'''

In [ ]:
keys = list(deltas.keys())
ini_key = keys[0]
seq_ini = []
sequence = []
data = []

def tree_test(k, seq_ini):
    for v in deltas[k]:
        sequence = seq_ini.copy()
        
        if len(seq_ini) != 0:
            if sequence[-1] > v:
                continue
            else:
                sequence.append(v)
        else:
            sequence.append(v)
        
        if len(sequence) == len(deltas.items()):
                data.append(sequence)
                #print(sequence)
        else:
                tree_test(str(int(k)+1), sequence)

        

tree_test(ini_key,sequence)
  

In [ ]:
print(len(data))T
print(data[-10:])

# Write output csv file

In [ ]:
from google.colab import files

#import file csv
uploaded = files.upload()


In [ ]:
import csv
import random
random.seed(227)

file_csv = open('test.csv', 'w', newline='')

filewriter = csv.writer(file_csv, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
filewriter.writerow([1,2,3,4,5,6])
for i in range(int(len(data)/2), len(data)):
    filewriter.writerow(data[i])

#files.download('train.csv')

In [ ]:
file_csv.close()
files.download('test.csv')


# LSTM model

In [ ]:
Tx, Ty =  6, 1

LSTM_model = model((Tx,1),Ty)
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

LSTM_model.load_weights('sources/LSTM_sequence_weights.h5')


# Band centers

In [ ]:
center_min, center_max  = [759., 769.]
center_max += 0.5

In [ ]:
path_search = 'Search/'
band_centers_ini = [str(int(value*10)) for value in arange(center_min, center_max, 0.5)]
band_centers = ['.{}0000000E+03'.format(str(int(value*10))) for value in arange(center_min, center_max, 0.5)]

current_path = os.getcwd()

In [ ]:
os.chdir(current_path + path_search)
#print(os.getcwd())
calculate_RES_for_band_centers(band_centers)
os.chdir('../')

# Clear res file

In [ ]:
res_path = 'Search/RES_files/'
for center in band_centers_ini:
    clear_res_file(res_path + 'TRANS_'+center+'.RES')

# Asym and search files

In [15]:
seria_name, quan_num = 'J1J-1', [1, 1, 1]
peak_file = 's'

In [12]:
path_search = 'Search/'
current_path = os.getcwd()

res_file = 'Search/RES_files/TRANS_7640.RES'

shutil.copyfile(res_file,  os.getcwd()+'/' + path_search +'TRANS.RES')

os.chdir(current_path +'/' + path_search)
make_asym_search(peak_file, quan_num)
os.chdir('../')

# Make sequences and PREDICTION 

In [ ]:
keys = list(deltas.keys())
ini_key = keys[0]
seq_ini = []
sequence = []
data = []

def tree_2(k, seq_ini):
    #print(k)
    for v in deltas[k]:
        #print(v)
        sequence = seq_ini.copy()
        
        sequence.append(v)
        #print(sequence, seq_ini)
    
        if len(sequence) == len(deltas.items()):
                sequence.append(0)
                data.append(sequence)
                #print(sequence)
        else:
                tree_2(str(int(k)+1), sequence)
        

tree_2(ini_key,sequence)

# Calculate deltas

In [ ]:
def deltas_in_search(file_name):
    file = open(file_name, 'r')

    count = 0
    for line in file:
        count +=1
        if count == 5:
            break
        continue

    deltas = {}
    upper = {}
    
    for line in file:
        if line.find('Searching') != -1:
            J0 = line.split()[1]
            deltas[J0] = []
            E0 = float(line.split()[-1])
            continue

        if line.find('No') != -1 or \
           len(line.split()) == 0 or \
           int(line.split()[0]) != int(J0) - 1:
            continue
        
        try:
            E = float(line.split()[-1])
            delta = round(E - E0, 3)
            deltas[J0].append(delta)
        except ValueError:
            break
    return deltas, indexes
